<a href="https://colab.research.google.com/github/y001003/system_quality_project/blob/main/data/%EB%B6%88%EB%A7%8C%EC%98%88%EC%B8%A1_%ED%95%84%EC%82%AC_4%EB%B2%88_%EC%A0%95%ED%98%95%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%8A%94_AutoML%EB%B6%80%ED%84%B0_%EB%8F%8C%EB%A0%A4%EB%B3%B4%EC%9E%90!_(%ED%8C%80%EC%9B%90_%EA%B5%AC%ED%95%A9%EB%8B%88%EB%8B%A4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycaret
!pip install jinja
!pip install markupsafe==2.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320 kB 5.2 MB/s 
     |████████████████████████████████| 88 kB 5.5 MB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 
     |████████████████████████████████| 120 kB 60.3 MB/s 
     |████████████████████████████████| 2.0 MB 48.0 MB/s 
     |████████████████████████████████| 167 kB 51.4 MB/s 
     |████████████████████████████████| 1.7 MB 44.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 17.8 MB 903 kB/s 
     |████████████████████████████████| 6.8 MB 36.3 MB/s 
     |████████████████████████████████| 1.3 MB 51.5 MB/s 
     |████████████████████████████████| 262 kB 59.3 MB/s 
     |████████████████████████████████| 1.3 MB 39.3 MB/s 
     |████████████████████████████████| 6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 252 kB 5.0 MB/s 
     |████████████████████████████████| 237 kB 54.5 MB/s 
     |████████████████████████████████| 120 kB 59.5 MB/s 
     |████████████████████████████████| 55 kB 4.2 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
ERROR: Could not find a version that satisfies the requirement jinja (from versions: 0.8, 0.9, 1.0, 1.1, 1.2)
ERROR: No matching distribution found for jinja
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.1
    Uninstalling MarkupSafe-2.1.1:
      Successfully uninstalled MarkupSafe-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling

In [2]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')




/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [5]:
from google.colab import drive
drive.mount('/content/drive')
PATH = '/content/drive/MyDrive/project/data/system_quality_customer_complain_data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
train_err  = pd.read_csv(PATH+'train_err_data_.csv')
id_error = train_err[['user_id','errtype']].values
id_error

array([[10000,    15],
       [10000,    12],
       [10000,    11],
       ...,
       [24999,     4],
       [24999,     4],
       [24999,    15]])

In [29]:
#error 0으로 이루어진 데이터셋 생성
error = np.zeros((15000,42)) # 15000개 user_id, 41가지(maxnum 42) errtype 

for person_idx, err in tqdm(id_error):
    # (person_idx - 10000)에 해당하는 행에, 열 기준 errtype 값의 -1 에 해당하는 위치에 error값을 +1
    error[person_idx - 10000,err - 1] += 1
error 

100%|██████████| 16554663/16554663 [00:41<00:00, 398089.25it/s]


array([[  0.,   0.,   8., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ..., 113.,  56.,   1.],
       [  0.,   0.,   2., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,  58.,   8.,   5.],
       [  0.,   0.,   0., ...,   6.,   0.,   0.],
       [  0.,   0.,   4., ...,   0.,   0.,   0.]])

In [30]:
# 불만을 제기한 사람 타겟값 생성
train_prob = pd.read_csv(PATH+'train_problem_data_.csv')
problem = np.zeros(15000)
problem[train_prob.user_id.unique()-10000] = 1 

# 타겟값을 포함한 학습 데이터 완성
train = pd.DataFrame(data=error)
train['problem'] = problem
train
# del error, problem



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,problem
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,1.0
2,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0,0.0
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,16.0,18.0,4.0,1.0,181.0,138.0,0.0,0.0,0.0,0.0,0.0,50.0,29.0,0.0,0.0,49.0,0.0,0.0,0.0,0.0,223.0,0.0,16.0,17.0,0.0,1.0,1.0,0.0,0.0,58.0,8.0,5.0,1.0
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,1.0,51.0,12.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,14.0,0.0,15.0,12.0,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0,1.0


In [27]:
a = pd.DataFrame(id_error)
a[0].drop_duplicates().reset_index()
error[0]
a[1].drop_duplicates().reset_index()[1].max()

42

In [31]:
# 모델 학습
clf = setup(data = train, target = 'problem')

,Description,Value
0,session_id,162
1,Target,problem
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 43)"
5,Missing Values,False
6,Numeric Features,42
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [32]:
# 모델 비교
best_5 = compare_models(sort = 'Accuracy', n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.7795,0.7978,0.4758,0.7685,0.5874,0.4482,0.4723,1.374
gbc,Gradient Boosting Classifier,0.7769,0.7996,0.4420,0.7900,0.5666,0.4323,0.4654,1.795
lightgbm,Light Gradient Boosting Machine,0.7761,0.7961,0.4830,0.7502,0.5873,0.4435,0.4639,0.291
rf,Random Forest Classifier,0.7726,0.7956,0.4588,0.7573,0.5708,0.4288,0.4540,1.736
ada,Ada Boost Classifier,0.7692,0.7855,0.4443,0.7566,0.5593,0.4172,0.4445,0.477
lr,Logistic Regression,0.7518,0.7334,0.3650,0.7570,0.4912,0.3534,0.3948,1.591
lda,Linear Discriminant Analysis,0.7483,0.7401,0.3206,0.7944,0.4564,0.3294,0.3869,0.080
ridge,Ridge Classifier,0.7473,0.0000,0.3108,0.8031,0.4478,0.3235,0.3849,0.018
qda,Quadratic Discriminant Analysis,0.7460,0.7391,0.3310,0.7696,0.4600,0.3282,0.3789,0.050
nb,Naive Bayes,0.7083,0.7285,0.1918,0.7201,0.3019,0.1899,0.2567,0.030


CatBoost, Lightgbm, gbc, et,rf 가 골고루 AUC 0.8대를 보여주기 때문에 5개를 골라 블렌딩 해보겠습니다

In [33]:
# 성능 좋은 모델 앙상블(블랜딩)
blended = blend_models(estimator_list = best_5, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7681,0.7864,0.4271,0.7668,0.5487,0.4092,0.4409
1,0.7933,0.8205,0.4841,0.8155,0.6076,0.4794,0.5095
2,0.7800,0.8003,0.4726,0.7736,0.5868,0.4485,0.4738
3,0.7943,0.8212,0.4885,0.8149,0.6108,0.4827,0.5119
4,0.7737,0.7917,0.4502,0.7685,0.5678,0.4283,0.4564
Mean,0.7819,0.8040,0.4645,0.7879,0.5843,0.4496,0.4785
Std,0.0104,0.0144,0.0229,0.0224,0.0236,0.0285,0.0283


In [34]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7814,0.8129,0.4739,0.8031,0.5961,0.4595,0.4896


In [35]:
final_model = finalize_model(blended)

In [ ]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((14999,42))
for person_idx, err in tqdm(id_error):
    test_x[person_idx - 30000,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
test = pd.DataFrame(data=test_x)

100%|██████████| 16532648/16532648 [00:42<00:00, 390642.21it/s]


In [ ]:
predictions = predict_model(final_model, data = test)

In [ ]:
#pycaret에서는 score이 label을 맞출 확률이기때문에 output을 제출 양식에 맞게 바꿔줍니다
x = []
for i in range(len(predictions['Score'])):
  if predictions['Label'][i] =='1.0':
    x.append(predictions['Score'][i])
  else:
    x.append(1-predictions['Score'][i])

In [ ]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')
sample_submssion['problem'] = x
sample_submssion.to_csv("AutoML.csv", index = False)